In [1]:
import tensorflow as tf
import numpy as np

import warnings
warnings.filterwarnings('ignore')

In [2]:
config = tf.ConfigProto()
config.log_device_placement = True

In [3]:
n_inputs = 28 * 28
n_hidden1 = 300
n_hidden2 = 100
n_outputs = 10

In [4]:
X = tf.placeholder(tf.float32, shape=(None, n_inputs), name="X")
y = tf.placeholder(tf.int64, shape=(None), name="y")

#### Build Computation Graph
* remove activation from layer, perform batch norm before activation
* use partial function to stay DRY
* drop out regularization

In [5]:
def max_norm_regularizer(threshold, axes=1, name="max_norm", collection="max_norm"):
    def max_norm(weights):
        clipped = tf.clip_by_norm(weights, clip_norm=threshold, axes=axes)
        clip_weights = tf.assign(weights, clipped, name=name)
        tf.add_to_collection(collection, clip_weights)
        return None # no regularization term
    return max_norm

In [6]:
max_norm_reg = max_norm_regularizer(threshold=1)

In [7]:
with tf.name_scope("dnn"):
    # dense layer uses xavier initialization by default, can change to he initialization
    he_init = tf.contrib.layers.variance_scaling_initializer()
    hidden1 = tf.layers.dense(X, n_hidden1, name="hidden1", 
                              kernel_initializer=he_init, activation=tf.nn.relu)
    hidden2 = tf.layers.dense(hidden1, n_hidden2, name="hidden2", 
                              activation=tf.nn.relu, kernel_regularizer=max_norm_reg)
    
    logits = tf.layers.dense(hidden2, n_outputs, name="outputs")


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Use keras.layers.dense instead.
Instructions for updating:
Colocations handled automatically by placer.


In [8]:
with tf.name_scope("loss"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
    loss = tf.reduce_mean(xentropy, name="loss")

In [9]:
learning_rate = 0.01

with tf.name_scope("train"):
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    training_op = optimizer.minimize(loss)

In [10]:
with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

In [11]:
init = tf.global_variables_initializer()
saver = tf.train.Saver()

#### Execution Phase

In [12]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/")

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /tmp/data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [13]:
n_epochs = 5
batch_size = 50

In [14]:
clip_all_weights = tf.get_collection("max_norm")

In [15]:
with tf.Session(config=config) as sess:
    init.run()
    for epoch in range(n_epochs):
        for interaction in range(mnist.train.num_examples // batch_size):
            X_batch, y_batch = mnist.train.next_batch(batch_size)
            feed = {X: X_batch, y: y_batch}
            sess.run(training_op, feed_dict=feed)
            sess.run(clip_all_weights)
        acc_train = accuracy.eval(feed_dict=feed)
        
        acc_val = accuracy.eval(feed_dict={X: mnist.validation.images, y: mnist.validation.labels})
        print("epoch %i, train acc %.5f, val acc %.5f"%(epoch, acc_train, acc_val))
    saver.save(sess, "outputs/final_model.ckpt")

epoch 0, train acc 0.98000, val acc 0.90380
epoch 1, train acc 0.92000, val acc 0.92160
epoch 2, train acc 0.84000, val acc 0.93100
epoch 3, train acc 0.98000, val acc 0.93780
epoch 4, train acc 0.96000, val acc 0.94460


#### Test

In [16]:
with tf.Session(config=config) as sess:
    saver.restore(sess, "outputs/final_model.ckpt")
    Z = logits.eval(feed_dict={X: mnist.test.images})
    acc_test = accuracy.eval(feed_dict={X: mnist.test.images, y: mnist.test.labels})
acc_test

Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from outputs/final_model.ckpt


0.9407